In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Etape 1:
1/ Importation des données.
2/ "URL", "RESPONSE", "HTML" permettent de récupérer le texte du site.
3/ "SOUP" permet d'utiliser les données pour les scraper.

In [2]:
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
data = soup.find_all('div')

### Etape 2:
1/ Création du dataframe "df1" recueillant toutes les données.
1.1/ "a" est le dataframe pour la phase de la compétition.
1.2/ "b" est le dataframe pour la compétition.
2/ Les deux dataframes "a" et "b" sont fusionnés dans le df1.

In [3]:
df1 = pd.DataFrame()

a = pd.DataFrame(columns=["Compétition"])
b = pd.DataFrame(columns=["Phase"])

col = soup.find_all('div', class_='match-item-event-series text-of')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    a = a._append({'Compétition':text1}, ignore_index=True)
col1 = soup.find_all('div', class_='match-item-event text-of')
for cols in col1:
    texts1 = cols.find_all(string=True, recursive=False)
    text2 = texts1[1].strip()
    b = b._append({'Phase':text2}, ignore_index=True)

df1 = b.join(a, how='inner')

In [4]:
df2 = pd.DataFrame()
c = pd.DataFrame(columns=["Team 2"])
d = pd.DataFrame(columns=["Team 1"])
col = soup.find_all('div', class_='match-item-vs-team-name')
for cols in col:
    texts = cols.find_all(string=True, recursive=True)
    text = texts[2].strip()
    text1 = texts[3].strip()
    d = d._append({'Team 1':text}, ignore_index=True)
    c = c._append({'Team 2':text1}, ignore_index=True)

c = c.replace('', np.nan, regex=True)
c = c.dropna()
c = c.reset_index(drop=True)
d = d.replace('', np.nan, regex=True)
d = d.dropna()
d = d.reset_index(drop=True)

df2 = d.join(c, how='inner')
df1 = df1.join(df2, how='inner')

In [5]:
col = soup.find_all('div', class_='match-item-vs-team-score js-spoiler')
df3 = pd.DataFrame()
e = pd.DataFrame(columns=["Score Team 1"])
f = pd.DataFrame(columns=["Score Team 2"])
    
for i, cols in enumerate(col):
    texts = cols.find_all(string=True, recursive=False)
    team = texts[0].strip()
    
    # Ajouter l'élément à 'c'
    if i % 2 == 0:
        e = e._append({'Score Team 1': team}, ignore_index=True)

for i, cols in enumerate(col):
    texts = cols.find_all(string=True, recursive=False)
    team = texts[0].strip()
    
    # Ajouter l'élément à 'c_impairs'
    if i % 2 != 0:
        f = f._append({'Score Team 2': team}, ignore_index=True)

df3 = e.join(f, how='inner')
df1 = df1.join(df3, how='inner')

In [6]:
g = pd.DataFrame(columns=["Date"])

col = soup.find_all('div', class_='ml-eta mod-completed')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    if 'd' in text1:
        jour, heures = map(int, text1.replace('d','').replace('h', '').replace('m', '').split())
        duree = timedelta(days=jour, hours=heures, minutes=minutes)
        time = datetime.today() - duree
        timeM = time.minute
        timeMsup = (timeM + 1) % 60
        diff = timedelta(minutes=timeMsup - timeM)
        time = time + diff
        g = g._append({'Date':time.strftime("%d/%m/%Y %Hh%M")}, ignore_index=True)
    else:
        heures, minutes = map(int, text1.replace('d','').replace('h', '').replace('m', '').split())
        duree = timedelta(hours=heures, minutes=minutes)
        time = datetime.today() - duree
        timeM = time.minute
        timeMsup = (timeM + 1) % 60
        diff = timedelta(minutes=timeMsup - timeM)
        time = time + diff
        g = g._append({'Date':time.strftime("%d/%m/%Y %Hh%M")}, ignore_index=True)

df1 = df1.join(g, how='inner')
df1

,Phase,Compétition,Team 1,Team 2,Score Team 1,Score Team 2,Date
0,Project V 2023: THE POKAL,Playoffs–Round of 16,ALTERNATE aTTaX,GEEKZ ENERGY eSPORTS,1,2,24/10/2023 17h00
1,Crossfire Cup 2023,Contenders #2 Playoffs–Quarterfinals,Team Galaga,Ramboot Club,2,0,24/10/2023 16h00
2,Superdome 2023: Egypt,Levant and North Africa Online Qualifier–Round 1,Geekay Esports,Team RA'AD,2,0,24/10/2023 16h00
3,China Evolution Series Act 2: Selection,Group Stage–Decider (B),FunPlus Phoenix,Rare Atom,2,0,24/10/2023 13h15
4,China Evolution Series Act 2: Selection,Group Stage–Decider (A),Bilibili Gaming,Trace Esports,0,2,24/10/2023 10h00
5,Sentinels Off-Season $5k Bounty Showmatch,Showmatch–Showmatch,Sentinels,Moist x Shopify,1,2,24/10/2023 02h30
6,Crossfire Cup 2023,Contenders #2 Playoffs–Quarterfinals,Movistar Riders,DSYRE,2,0,23/10/2023 20h30
7,Superdome 2023: Egypt,GCC And Iraq Online Qualifier–Round 1,Triple Esports Club,Twisted Minds Esports,2,0,23/10/2023 20h30
8,Crossfire Cup 2023,Contenders #2 Playoffs–Quarterfinals,Team Aku Aku,UCAM Tokiers,2,1,23/10/2023 17h30
9,Superdome 2023: Egypt,GCC And Iraq Online Qualifier–Round 1,The Ultimates,Team Falcons,0,2,23/10/2023 17h30


In [7]:
df1.Date + timedelta(seconds=15)

TypeError: unsupported operand type(s) for +: 'Timedelta' and 'str'